# TensorFlow Serving

## Docker example

### Installazione

_docker pull tensorflow/serving_

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from random import randint

### Preparazione fake data e del modello

In [4]:
X = [randint(0,100) for i in range(1000)]

In [5]:
y = [x*2+x/2 for x in X]

In [38]:
with tf.device('/cpu:0'):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(100, input_shape=(1,)))
    model.add(keras.layers.Dense(1))

    model.compile(loss='mae', optimizer='adam', metrics='mae')
    
    model.fit(X,y, epochs=100, verbose=0)

In [21]:
with tf.device('/cpu:0'):
    y_hat = model.predict([10])

In [22]:
y_hat

array([[24.959215]], dtype=float32)

### Salvataggio del modello

In [23]:
# Nome del modello
name = 'example'

# Versione del modello (TensorFlow Serving gestisce automaticamente l'ultima versione)
version = '001'

# Salvataggio
model.save(filepath=f'models/{name}/{version}', save_format='pb')

INFO:tensorflow:Assets written to: models/example/003/assets


### Avvio del modello attraverso un'istanza docker

**path dei modelli**

_MODEL_DIR="$(pwd)/models"_

All'interno di questa cartella ci sarà dunque ogni singolo modello in una cartella (example in questo caso), all'interno di ogni cartella ci sarà dunque una sottocartella numerica che conterrà la versione del modello.

**avvio dell'istanza docker per il modello _example_**

docker run -t --rm -p 8501:8501 -v "$MODEL_DIR/example:/models/example" -e MODEL_NAME=example tensorflow/serving &

**esempi di interrogazione attraverso curl**

curl -d "{\"instances\": [[10]]}" -X POST http://localhost:8501/v1/models/example:predict

{
    "predictions": [[24.9380608]
    ]
}

curl -d "{\"instances\": [[10], [10], [20]]}" -X POST http://localhost:8501/v1/models/example:predict

### Interrogazione API attraverso la libreria "requests"

In [11]:
import requests

In [12]:
data = dict()

In [24]:
data['instances'] = [[i] for i in range(9, 11)]
data

{'instances': [[9], [10]]}

In [33]:
url = "http://localhost:8501/v1/models/example:predict"

In [34]:
response = requests.post(url, json=data)

In [35]:
response.status_code

200

In [36]:
response.json()

{'predictions': [[22.5051498], [25.0058746]]}